### Gradiente por descenso
#### Equipo No. 6: Elena Villalobos Nolasco, Carolina Acosta Tovany y Aide Jazmín González Cruz

#### 1.- Se importan los paquetes

In [1]:
import random
import csv
import numpy as np
import pandas as pd

#### 2.- Cargamos los datos

In [2]:
def load_data(path, header):
    marks_df = pd.read_csv(path, header=header)
    return marks_df

data = load_data("iris_num.txt",None)
data = np.array(data)

#### 3.- Definimos nuestra función costo/objetivo (LOSS Function)

$J(\theta,\theta_0) = \frac{1}{n} \sum_{i=1}^{n} L_h (y^{(i)}(\theta^{T}x^{i} + \theta_0)) + \frac{\lambda}{2} || \theta || ^2$

In [3]:
def costo(teta,teta0,data,n,lamda):#Función Objetivo J(Ø,Ø0)
    suma = np.array([0.0])
    yi = 0.0
    modulo = 0.0
    modulo = (teta[0]*teta[0]) + (teta[1]*teta[1]) + (teta[2]*teta[2]) + (teta[3]*teta[3])
    
    for i in data:
        if(i[4] == 0.0):
            yi = -1.0
        else:
            yi = 1.0
                
        x = i[:len(i)-1]       
              
        ##(yi(ØTxi + Ø0))
        v = yi * ((np.dot(np.array(teta).transpose(),np.array(x))) + teta0)
        
        ##SUMA Lh(yi(ØTxi + Ø0))
        if v < 1.0:
            Lh = (1.0 - v)
        else:
            Lh = 0.0
            
        suma =  suma + (Lh + ((lamda/2)*modulo))
        
    return ((1/n)*suma)


#### 5.- Definimos nuestra función descenso por gradiente

In [4]:
def descenso_x_gradiente(data, eta, Oini, O0ini, lamda, epsilon):
        # Se crea arreglo para guardar los pesos que se van calculando               
        O = []
        O0 = []
        O.append(Oini)
        O0.append(O0ini)
        
        Ot = [0.0, 0.0, 0.0, 0.0]
        Ot0 = [0.0]
        
        t = 0
        
        n = len(data)
        
        while True:  
            t = t + 1
            sumaO = np.array([0.0, 0.0, 0.0, 0.0])
            sumaO0 = np.array([0.0])
            
            for i in data:
                
                if(i[4] == 0.0):
                    yi = -1.0
                else:
                    yi = 1.0              
                                    
                x = i[:len(i)-1]
                
                ##Valida esta parte y(i)(Ø(t-1)Tx(i)+Ø0(t-1)) 
                hip = yi * ((np.dot(np.array(O[t-1]).transpose(),np.array(x))) + O0[t-1])
                
                val = 0.0
                if(hip < 1.0):
                    val = -1.0
                else:
                    val = 0.0
                    
                hip_O = (val * (yi * np.array(x))) + (lamda * O[t-1])
                hip_O0 = val * yi
                
                sumaO = sumaO + hip_O
                sumaO0 = sumaO0 + hip_O0                        
            
            Ot = O[t-1] - eta * ((1/n)*sumaO)
            Ot0 = O0[t-1] - eta * (((1/n)*sumaO0))
                                 
            O = np.insert(O, t, Ot, axis=0)
            O0 = np.insert(O0, t, Ot0, axis=0)
            
            cost = abs(costo(O[t],O0[t],data,n,lamda) - costo(O[t-1],O0[t-1],data,n,lamda))     
            
                                   
            #Se corre el while hasta alcanzar el epsilon o las iteraciones son mayor a 1000 (para no tender a loop infinito)    
            if((cost < epsilon) or (t > 1000)):
                break 
        
        #print('t: ' + str(t))      
        return [O[t], O0[t]]
        

#### 6.- Definimos nuestra función de validación cruzada

In [9]:
random.seed(1993)

def CrossValidation(data,K):
    lamda_list = [100,10,1,0.1]
    eta = 0.01 # LearningRate
    epsilon = 0.05
    df_lamda = pd.DataFrame(0.0, index=range(1),columns=("100","10","1","0.1"))
    
    # Dividimos los datos en chunk de igual tamaño
    chunks = np.array_split(data, K)
    for lamda_ in lamda_list:
        error = 0.0
        for i in range(1,int(K)):
            
            # Inicializamos las tetas
            Oini = np.array([0.0, 0.0, 0.0, 0.0])
            O0ini = np.array([0.0])
            
            # El chunk i será nuestro bloque de datos de prueba
            test_data = chunks[i]
            
            # Entrenamos con todos los otros datos menos con el chunk[i]
            train_data = chunks.copy()
            train_data.pop(i)
            train_data = np.concatenate(train_data)
            tetas = descenso_x_gradiente(train_data, eta, Oini, O0ini, lamda_, epsilon)
            
            #  Obtenemos el error con la función objetivo
            error += costo(tetas[0],tetas[1],test_data,len(test_data),lamda_)
        error /= K
        
        df_lamda[str(lamda_)][0] = error
        
    print(df_lamda)
    return (df_lamda.idxmin(axis=1)[0])


#### 7.- Hacemos la validación cruzada sobre nuestra data

In [6]:
df_lamda = CrossValidation(data,5.0)
float(df_lamda)

        100        10         1       0.1
0  0.720983  0.427988  0.277125  0.268085


0.1

#### Conclusión:

De acuerdo al CrossValidation , debemos utilizar lambda 0.1 para obtener mejor resultados, porque tendrá menor error segun la funcion de costo. 

Estamos cosiderando un eta de 0.01 y un epsilon de 0.05

#### 8.- Volvemos a entrenar

In [8]:
#Se revuelven los datos
random.shuffle(data)

#Datos de entrenamiento
datos_entrenamiento = data[0:120]

#Datos de verificación
data_validacion = data[121:150]


eta = 0.01
epsilon = 0.05
Oini = np.array([0.0, 0.0, 0.0, 0.0])
O0ini = np.array([0.0])

tetas = descenso_x_gradiente(datos_entrenamiento, eta, Oini, O0ini, float(df_lamda), epsilon)
print("Teta : ", tetas[0], " \nTeta_0:", tetas[1])

Teta :  [0.14013694 0.1001023  0.03228874 0.00318462]  
Teta_0: [0.029]
